In [25]:
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf

# Load the data set and get the parameters and labels 

In [26]:
weather_data = pd.read_csv('./data/SortedWeatherData_TorontoLabel.csv')

In [27]:
weather_data.head(5)

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main_x,weather_description,weather_icon,weather_main_y
0,1684281600,2023-05-17 00:00:00 +0000 UTC,-14400,Toronto,43.653226,-79.383184,15.14,10000.0,7.19,14.25,...,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01d,Clear
1,1684281600,2023-05-17 00:00:00 +0000 UTC,-14400,Peterborough,44.304706,-78.319961,10.39,10000.0,4.32,9.21,...,NaN,NaN,NaN,NaN,40,802,Clouds,scattered clouds,03d,Clear
2,1684281600,2023-05-17 00:00:00 +0000 UTC,-14400,Kitchener,43.451639,-80.492534,12.52,10000.0,7.41,11.68,...,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01d,Clear
3,1684281600,2023-05-17 00:00:00 +0000 UTC,-14400,Barrie,44.389356,-79.690332,10.00,NaN,3.73,7.03,...,NaN,NaN,NaN,NaN,78,803,Clouds,broken clouds,04d,Clear
4,1684281600,2023-05-17 00:00:00 +0000 UTC,-14400,Niagara Falls,43.089558,-79.084944,15.11,10000.0,7.41,14.24,...,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04d,Clear


# Get the unique characteristics of the climate

In [30]:
# There are different IDs for the climate condition
weather_category = weather_data.get(['weather_id', 'weather_main_x', 'weather_description']).drop_duplicates(subset=['weather_id', 'weather_main_x', 'weather_description'])
#Later we can use the IDs to confirm if the weather was correctly predicted. 
weather_category.head(30)

,weather_id,weather_main_x,weather_description
0,800,Clear,sky is clear
1,802,Clouds,scattered clouds
3,803,Clouds,broken clouds
7,804,Clouds,overcast clouds
10,801,Clouds,few clouds
330,500,Rain,light rain
364,501,Rain,moderate rain
368,721,Haze,haze
379,701,Mist,mist
409,502,Rain,heavy intensity rain


In [ ]:
weather_category.to_csv('./data/unique_weather_category.csv', index=False ,sep=',', header=True, encoding='utf-8')

# Preprocessing data and removing unnecessary features

In [29]:
X = weather_data.drop(['dt_iso','weather_main_y', 'weather_main_x', 'weather_description','weather_icon', 'timezone', 'sea_level', 'grnd_level'], axis=1)
y = weather_data["weather_main_y"]

In [16]:
X.head(10)

,dt,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,...,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main_x
0,1684281600,Toronto,43.653226,-79.383184,15.14,10000.0,7.19,14.25,14.43,15.61,...,7.72,330,12.86,NaN,NaN,NaN,NaN,0,800,Clear
1,1684281600,Peterborough,44.304706,-78.319961,10.39,10000.0,4.32,9.21,9.74,11.01,...,6.17,10,10.80,NaN,NaN,NaN,NaN,40,802,Clouds
2,1684281600,Kitchener,43.451639,-80.492534,12.52,10000.0,7.41,11.68,11.87,13.12,...,5.66,10,8.75,NaN,NaN,NaN,NaN,0,800,Clear
3,1684281600,Barrie,44.389356,-79.690332,10.00,NaN,3.73,7.03,8.06,10.81,...,6.69,10,9.77,NaN,NaN,NaN,NaN,78,803,Clouds
4,1684281600,Niagara Falls,43.089558,-79.084944,15.11,10000.0,7.41,14.24,13.33,16.12,...,3.60,350,NaN,NaN,NaN,NaN,NaN,75,803,Clouds
5,1684285200,Kitchener,43.451639,-80.492534,9.28,10000.0,5.27,6.27,8.21,11.34,...,6.17,350,NaN,NaN,NaN,NaN,NaN,0,800,Clear
6,1684285200,Niagara Falls,43.089558,-79.084944,12.21,10000.0,7.11,11.34,11.28,12.68,...,6.17,340,9.77,NaN,NaN,NaN,NaN,40,802,Clouds
7,1684285200,Barrie,44.389356,-79.690332,8.01,NaN,2.05,4.48,7.10,8.88,...,6.69,360,NaN,NaN,NaN,NaN,NaN,95,804,Clouds
8,1684285200,Toronto,43.653226,-79.383184,12.97,10000.0,5.40,11.89,12.21,13.38,...,6.17,320,10.80,NaN,NaN,NaN,NaN,0,800,Clear
9,1684285200,Peterborough,44.304706,-78.319961,8.14,10000.0,2.18,4.49,7.52,9.22,...,7.20,10,9.77,NaN,NaN,NaN,NaN,75,803,Clouds


,dt,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,...,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main_x
0,1684281600,Toronto,43.653226,-79.383184,15.14,10000.0,7.19,14.25,14.43,15.61,...,7.72,330,12.86,NaN,NaN,NaN,NaN,0,800,Clear
1,1684281600,Peterborough,44.304706,-78.319961,10.39,10000.0,4.32,9.21,9.74,11.01,...,6.17,10,10.80,NaN,NaN,NaN,NaN,40,802,Clouds
2,1684281600,Kitchener,43.451639,-80.492534,12.52,10000.0,7.41,11.68,11.87,13.12,...,5.66,10,8.75,NaN,NaN,NaN,NaN,0,800,Clear
3,1684281600,Barrie,44.389356,-79.690332,10.00,NaN,3.73,7.03,8.06,10.81,...,6.69,10,9.77,NaN,NaN,NaN,NaN,78,803,Clouds
4,1684281600,Niagara Falls,43.089558,-79.084944,15.11,10000.0,7.41,14.24,13.33,16.12,...,3.60,350,NaN,NaN,NaN,NaN,NaN,75,803,Clouds
5,1684285200,Kitchener,43.451639,-80.492534,9.28,10000.0,5.27,6.27,8.21,11.34,...,6.17,350,NaN,NaN,NaN,NaN,NaN,0,800,Clear
6,1684285200,Niagara Falls,43.089558,-79.084944,12.21,10000.0,7.11,11.34,11.28,12.68,...,6.17,340,9.77,NaN,NaN,NaN,NaN,40,802,Clouds
7,1684285200,Barrie,44.389356,-79.690332,8.01,NaN,2.05,4.48,7.10,8.88,...,6.69,360,NaN,NaN,NaN,NaN,NaN,95,804,Clouds
8,1684285200,Toronto,43.653226,-79.383184,12.97,10000.0,5.40,11.89,12.21,13.38,...,6.17,320,10.80,NaN,NaN,NaN,NaN,0,800,Clear
9,1684285200,Peterborough,44.304706,-78.319961,8.14,10000.0,2.18,4.49,7.52,9.22,...,7.20,10,9.77,NaN,NaN,NaN,NaN,75,803,Clouds


In [11]:
X.head(10)

,dt,city_name,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,...,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main_x
0,1684281600,Toronto,43.653226,-79.383184,15.14,10000.0,7.19,14.25,14.43,15.61,...,7.72,330,12.86,NaN,NaN,NaN,NaN,0,800,Clear
1,1684281600,Peterborough,44.304706,-78.319961,10.39,10000.0,4.32,9.21,9.74,11.01,...,6.17,10,10.80,NaN,NaN,NaN,NaN,40,802,Clouds
2,1684281600,Kitchener,43.451639,-80.492534,12.52,10000.0,7.41,11.68,11.87,13.12,...,5.66,10,8.75,NaN,NaN,NaN,NaN,0,800,Clear
3,1684281600,Barrie,44.389356,-79.690332,10.00,NaN,3.73,7.03,8.06,10.81,...,6.69,10,9.77,NaN,NaN,NaN,NaN,78,803,Clouds
4,1684281600,Niagara Falls,43.089558,-79.084944,15.11,10000.0,7.41,14.24,13.33,16.12,...,3.60,350,NaN,NaN,NaN,NaN,NaN,75,803,Clouds
5,1684285200,Kitchener,43.451639,-80.492534,9.28,10000.0,5.27,6.27,8.21,11.34,...,6.17,350,NaN,NaN,NaN,NaN,NaN,0,800,Clear
6,1684285200,Niagara Falls,43.089558,-79.084944,12.21,10000.0,7.11,11.34,11.28,12.68,...,6.17,340,9.77,NaN,NaN,NaN,NaN,40,802,Clouds
7,1684285200,Barrie,44.389356,-79.690332,8.01,NaN,2.05,4.48,7.10,8.88,...,6.69,360,NaN,NaN,NaN,NaN,NaN,95,804,Clouds
8,1684285200,Toronto,43.653226,-79.383184,12.97,10000.0,5.40,11.89,12.21,13.38,...,6.17,320,10.80,NaN,NaN,NaN,NaN,0,800,Clear
9,1684285200,Peterborough,44.304706,-78.319961,8.14,10000.0,2.18,4.49,7.52,9.22,...,7.20,10,9.77,NaN,NaN,NaN,NaN,75,803,Clouds


In [8]:
X.fillna(X.mean(), inplace=True)

TypeError: Could not convert ['TorontoPeterboroughKitchenerBarrieNiagara FallsKitchenerNiagara FallsBarrieTorontoPeterboroughNiagara FallsKitchenerTorontoBarriePeterboroughTorontoPeterboroughBarrieNiagara FallsKitchenerKitchenerBarriePeterboroughTorontoNiagara FallsBarriePeterboroughNiagara FallsKitchenerTorontoTorontoPeterboroughNiagara FallsBarrieKitchenerBarriePeterboroughTorontoNiagara FallsKitchenerBarrieNiagara FallsKitchenerPeterboroughTorontoNiagara FallsKitchenerPeterboroughTorontoBarriePeterboroughBarrieTorontoKitchenerNiagara FallsPeterboroughTorontoKitchenerNiagara FallsBarrieBarrieNiagara FallsPeterboroughTorontoKitchenerNiagara FallsPeterboroughTorontoKitchenerBarriePeterboroughKitchenerNiagara FallsBarrieTorontoNiagara FallsBarriePeterboroughTorontoKitchenerBarriePeterboroughKitchenerTorontoNiagara FallsBarriePeterboroughKitchenerTorontoNiagara FallsTorontoPeterboroughBarrieNiagara FallsKitchenerTorontoKitchenerBarriePeterboroughNiagara FallsPeterboroughBarrieNiagara Fal

In [ ]:
categorical_features = ["city_name", "weather_main"]
numerical_features = ["dt_iso", "lat", "lon", "temp", "visibility","dew_point","feels_like","temp_min","temp_max","pressure","humidity","wind_speed","wind_deg","wind_gust","rain_1h","rain_3h","snow_1h","snow_3h","clouds_all","weather_main_x"]

In [ ]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
X_processed = preprocessor.fit_transform(X)

In [ ]:
y_tensor = tf.convert_to_tensor(y)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(100, activation='relu', input_shape=[X_processed.shape[1]]),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
model.summary()